In [1]:
import torch
from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision
import matplotlib.pyplot as plt
import torch.functional as F
%matplotlib inline

In [2]:
training_data = [
    ("The dog ate the apple".split(),["DET","NN","V","DET","NN"]),
    ("Everybody read that book".split(),["NN","V","DET","NN"])
]



In [3]:
word_to_idx = {}
tag_to_idx = {}
idx_to_tag = {}
for context,tag in training_data:
    for word in context:
        if word.lower() not in word_to_idx:
            word_to_idx[word.lower()] = len(word_to_idx)
    for label in tag:
        if label not in tag_to_idx:
            tag_to_idx[label] = len(tag_to_idx)
            idx_to_tag[len(tag_to_idx)-1] = label
            
alphabet = 'abcdefghijklmnopqrstuvwxyz'
char_to_idx = {}
for i in range(len(alphabet)):
    char_to_idx[alphabet[i]] = i

In [4]:
class CharLSTM(nn.Module):
    '''
    构建单个字符分类器，对单词的字符判断词性，比如ly为副词
    将隐含状态的最后一个输出
    对于每个单词，都可以通过CharLSTM用相应的字符表示
    '''
    def __init__(self,n_char,char_dim,char_hidden):
        super(CharLSTM,self).__init__()
        self.char_embedding = nn.Embedding(n_char,char_dim)
        self.char_lstm = nn.LSTM(char_dim,char_hidden,batch_first=True)
    def forward(self,x):
        x = self.char_embedding(x)
        _,h = self.char_lstm(x)
        return h[0]
    

In [5]:
class LSTMTagger(nn.Module):
    def __init__(self,n_word,n_char,char_dim,word_dim,char_hidden,word_hidden,n_tag):
        super(LSTMTagger,self).__init__()
        self.word_embedding = nn.Embedding(n_word,word_dim)
        self.char_lstm = CharLSTM(n_char,char_dim,char_hidden)
        self.word_lstm = nn.LSTM(word_dim+char_hidden,word_hidden,batch_first=True)
        self.linear1 = nn.Linear(word_hidden,n_tag)
    def forward(self, x, word):
        char =[]
        for w in word:
            char_list = [char_to_idx[i.lower()] for i in w]
            char_list = torch.LongTensor(char_list)
            char_list = char_list.unsqueeze(0)
            tempchar = self.char_lstm(Variable(char_list))
            tempchar = tempchar.squeeze(0)
            char.append(tempchar)
 
        char = torch.stack(char,dim=0)
        x = self.word_embedding(x) # batch,seq,word_dim
#         print(x.size(),char.size())
        x = x.permute(1,0,2)
        x = torch.cat((x,char),dim=2)
        x,_ = self.word_lstm(x)#seq,batch,word_hidden
        
        s,b,h = x.shape
        x = x.view(-1,h)
        out = self.linear1(x)
#         out = F.log_softmax(out)
        return out         
        

In [6]:
lstm = LSTMTagger(len(word_to_idx),len(char_to_idx),10,100,50,128,len(tag_to_idx))
optimizer = optim.Adam(lstm.parameters(),lr=0.02)
loss_func = nn.CrossEntropyLoss()

In [9]:
for epoch in range(300):
    train_loss = 0
    for word,tag in training_data:
        word_list = [word_to_idx[w.lower()] for w in word]
        word_list = torch.LongTensor(word_list).unsqueeze(0)
        tag = [tag_to_idx[t] for t in tag]
        tag = torch.LongTensor(tag)
        word_list = Variable(word_list)
        tag = Variable(tag)
        
        # forward
        pred_tag = lstm(word_list,word)
        loss = loss_func(pred_tag,tag)
        train_loss += loss.item()
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch+1)%50 == 0:
            print('Epoch:{},Loss:{:.5f}'.format(epoch+1,train_loss/len(training_data)))
lstm.eval()
test_sent = 'dog read the book'
test_list = [word_to_idx[w.lower()] for w in test_sent.split()]
test_list = torch.LongTensor(test_list).unsqueeze(0)
out = lstm(Variable(test_list),test_sent.split())
_,pred = torch.max(out,1)
print([idx_to_tag[int(i)] for i in pred])

Epoch:50,Loss:0.00000
Epoch:50,Loss:0.00000
Epoch:100,Loss:0.00000
Epoch:100,Loss:0.00000
Epoch:150,Loss:0.00000
Epoch:150,Loss:0.00000
Epoch:200,Loss:0.00000
Epoch:200,Loss:0.00000
Epoch:250,Loss:0.00000
Epoch:250,Loss:0.00000
Epoch:300,Loss:0.00000
Epoch:300,Loss:0.00000
['NN', 'V', 'DET', 'NN']
